In [1]:
## Normal Python Libraries

import matplotlib
matplotlib.use('Agg')
import matplotlib.pylab as plt
from scipy import signal
import numpy as np
from scipy import interpolate
import pandas as pd
from pyproj import Proj
import copy
import os
from os import path
from base64 import b64encode

## Import EikoNet Libraries

from EikoNet import model as md
from EikoNet import database as db
from EikoNet import plot as pt 

## Import pytorch Libraries

import torch
from torch.nn import Linear
from torch import Tensor
from torch.nn import MSELoss
from torch.optim import SGD, Adam, RMSprop
from torch.autograd import Variable, grad
from torch.utils.data.sampler import SubsetRandomSampler,WeightedRandomSampler

In [2]:
def _randPoints(numsamples=10000,randomDist=False,Xmin=[0,0,0],Xmax=[2,2,2]):
    numsamples = int(numsamples)
    Xmin = np.append(Xmin,Xmin)
    Xmax = np.append(Xmax,Xmax)
    if randomDist:
        X  = np.zeros((numsamples,6))
        PointsOutside = np.arange(numsamples)
        while len(PointsOutside) > 0:
            P  = np.random.rand(len(PointsOutside),3)*(Xmax[:3]-Xmin[:3])[None,None,:] + Xmin[:3][None,None,:]
            dP = np.random.rand(len(PointsOutside),3)-0.5
            rL = (np.random.rand(len(PointsOutside),1))*np.sqrt(np.sum((Xmax-Xmin)**2))
            nP = P + (dP/np.sqrt(np.sum(dP**2,axis=1))[:,np.newaxis])*rL

            X[PointsOutside,:3] = P
            X[PointsOutside,3:] = nP

            maxs          = np.any((X[:,3:] > Xmax[:3][None,:]),axis=1)
            mins          = np.any((X[:,3:] < Xmin[:3][None,:]),axis=1)
            OutOfDomain   = np.any(np.concatenate((maxs[:,None],mins[:,None]),axis=1),axis=1)
            PointsOutside = np.where(OutOfDomain)[0]
    else:
        X  = (np.random.rand(numsamples,6)*(Xmax-Xmin)[None,None,:] + Xmin[None,None,:])[0,:,:]
    return X

In [3]:
xmin = [-122.77,47.46,0]
xmax = [-122.29,47.93,50]
projection = "+proj=utm +zone=10, +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
Xp = _randPoints(numsamples=10000,randomDist=True,Xmin=xmin,Xmax=xmax)

In [6]:
Xp.shape
Yp = np.zeros((Xp.shape[0],2))
Yp[0]

array([0., 0.])

In [ ]:
#### Read in the cvm. 
df = pd.read_csv("/Users/banjo/uw/classes/bjohnson_repo/final_project/util/brem_cvm_eikonet_.csv")
df['z'] = df['z']/1000

In [ ]:
# df['z'] = df['z']/1000

In [ ]:
def eval(Xp):
    df = pd.read_csv("/Users/banjo/uw/classes/bjohnson_repo/final_project/util/brem_cvm_eikonet_.csv")
    df['z'] = df['z']/1000
    velocity = []
    Yp = np.zeros((Xp.shape[0],2))
    for i in range(len(Xp)): 
        t = Xp[i]
        x_dep = t[5]
        sub = df.iloc[(df['z']-x_dep).abs().argsort()[:2]]
        sub_z = np.unique(sub['z'].values)
        
        zdf = df.loc[(df['z'].values) == sub_z]
        proj = Proj(projection)
        t1 = proj(t[3],t[4])
        print("t1",t1)
        x_utme = t1[0]
        x_utmn = t1[1]
        

        sub = df.loc[(df['utmn']-x_utmn).abs().argsort()[:2]]
        sub_n = np.unique(sub['utmn'].values)

        sub = df.loc[(df['utme']-x_utme).abs().argsort()[:2]]
        sub_e = np.unique(sub['utme'].values)

        row = zdf.loc[(zdf['utme'].values==sub_e) & (zdf['utmn'].values== sub_n)]
        print(row)
#     vp = row['vp'].values[0]
#     velocity.append(vp)
# Yp[:,1] = velocity

In [ ]:
eval(Xp)

In [ ]:
# row = zdf.loc[((zdf['utme'].values) == sub_e) & ((zdf['utmn'].values)== sub_n)]
# vp = row['vp'].values
# vp = row['vp'].values[0]
# vp
# Yp
# t = x[1]
# t[5]
# input = t[2]
# sub = df.iloc[(df['z']-input).abs().argsort()[:2]]
# sub_z = np.unique(sub['z'].values)
# zdf = df.iloc[(df['z'].values) == sub_z]

In [ ]:
proj = Proj(projection)
t1 = proj(t[0],t[1])
# print(t1)
x_utme = t1[0]
x_utmn = t1[1]

sub = df.iloc[(df['utmn']-x_utmn).abs().argsort()[:2]]
sub_n = np.unique(sub['utmn'].values)
# sub_n
sub = df.iloc[(df['utme']-x_utme).abs().argsort()[:2]]
sub_e = np.unique(sub['utme'].values)
# sub_e

In [ ]:
row = zdf.loc[((zdf['utme'].values) == sub_e) & ((zdf['utmn'].values)== sub_n)]
Y = row['vp'].values

In [ ]:
t

In [ ]:
t = x[1]
input = t[1]
sub = df.iloc[(df['lon']-input).abs().argsort()[:2]]
sub_lon = np.unique(sub['lon'].values)

In [ ]:
w = x[1]
input = w[0]
sub = df.iloc[(df['lat']-input).abs().argsort()[:2]]
sub_lat = np.unique(sub['lat'].values)
t = x[1]
input = t[1]
sub = df.iloc[(df['lon']-input).abs().argsort()[:2]]
sub_lon = np.unique(sub['lon'].values)
q = x[1]
input = q[2]
sub = df.iloc[(df['z']-input).abs().argsort()[:2]]
sub_dep = np.unique(sub['z'].values)
# point = list(zip(sub_lat,sub_lon,sub_dep))
# row = df.iloc[((df['z'].values) == sub_dep)]
# rows = row.iloc[((row['lon'].values) == sub_lon)]
# rowz = rows.iloc[((rows['lat'].values) == sub_lat)]
# print(rows)
    

    
    # #                    & (df['z'].values == sub_dep)]
# df.loc[(df[‘Color’] == ‘Green’) & (df[‘Shape’] == ‘Rectangle’)]

In [ ]:
row

In [ ]:
sub_lon

In [ ]:
point

In [ ]:
df.loc[(df['lat'].index == sub_lat) & (df['lon'].index == sub_lon) & (df['z'].index == sub_dep)]

In [ ]:
q = x[1]
input = q[2]
df.iloc[(df['z']-input).abs().argsort()[:2]]

In [ ]:
point

In [ ]:
point = x[1]
# point[0]
X = point[0]
Y = point[1]
Z=point[2]


In [ ]:
a = df.to_numpy()
a_list = a[1][0:3]
given_value = x[1][0:3]
# a_list = (df['lat','lon','z']).to_numpy
absolute_difference_function = lambda list_value : abs(list_value - given_value)

closest_value = min(a_list, key=absolute_difference_function)
print(closest_value)


In [ ]:
a_list = a[1][0:3]
a_list

In [ ]:
x